# Code

## Imports

In [12]:
import inseq
import pandas as pd
import warnings
import byt5_model
import torch

warnings.filterwarnings("ignore")

## Functions

In [3]:
class InseqAttributer:
    def __init__(self, model="./models/test/", attribution_method="input_x_gradient") -> None:
        """Create an object that loads a given model through Inseq with a given attribution method"""
        self.model = inseq.load_model(model, attribution_method)
    
    def attribute(self, inp:str, out:str=None):
        """Use the Inseq model to generate feature attributions using a given input or input & output"""
        if inp and out:
            inp_out = (inp, out)
        else:
            inp_out = (inp,)

        inseq_out = self.model.attribute(*inp_out,
                                         attribute_target=True,
                                         step_scores=["probability"])
        inseq_out.show()

In [10]:
def data_sampler(data, model, samples=10, spaces=True, device='auto'):

    if device == 'auto':
        device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")
    elif device == 'gpu' or device == 'cuda':
        device = torch.device("cuda")
    elif device == 'cpu':
        device = torch.device("cpu")
    else:
        print('Set device to auto, cpu, or gpu/cuda.')
        return

    test = pd.read_csv(
        data,
        sep='\t',
        names=('lemma', 'labels', 'features')
    )

    if spaces:
        test['inputs'] = test['lemma'] + ' ' + test['features']
    else:
        test['inputs'] = test['lemma'] + test['features']


    test_gen_comparison = byt5_model.comparer(test, byt5_model.generate(model, test, device))
    test = pd.concat([test, test_gen_comparison], axis=1).rename(columns={'labels': 'plural'})

    correct = []
    for idx, row in test.iterrows():
        if row['Expected'] == row['Predicted']:
            correct.append('correct')
        else:
            correct.append('incorrect')

    test = test.assign(correct=correct)

    return pd.concat([test[test.correct == 'incorrect'].sample(n=samples//2, random_state=1234),
                      test[test.correct == 'correct'].sample(n=samples//2, random_state=1234)])

In [5]:
def sample_runner(samples, inseq_model):
    count = 0
    for idx, sample in samples.iterrows():
        count += 1
        if count > 1:
            print('\n')
        print('='*80)
        print('{} - {}: \'{}\' -> \'{}\''.format(
            count, idx, sample['lemma'] + ' ' + sample['features'], sample['plural']
        ))
        print('='*80)
        inseq_model.attribute(inp=sample['lemma'] + ' ' + sample['features'],
                              out=sample['plural'])

## German

### Setup

In [6]:
model_ger_finetuned = "./models/finetuned_ger"
inseq_ger_finetuned = InseqAttributer(model_ger_finetuned, "input_x_gradient")

### German Finetuned

The sample_runner function splits the given amount of samples in `two` (default is 10) so that `~50% of the samples are samples of correct predictions`, and `~50% of the samples are samples of incorrect predictions`.

First the incorrect predictions are shown, then the correct ones are shown (in the outcomes below, this translates to `5 incorrect items or 1-5, followed by 5 correct items or 6-10`).

In [7]:
sample_runner(data_sampler('./data/deu.gold', model_ger_finetuned), inseq_ger_finetuned)

        Expected     Predicted
0         Orgien        Orgien
1         Sieger        Sieger
2         Klötze        Klotze
3          Kalke         Kalke
4       Skelette      Skelette
5        Flocken       Flocken
6       Schwänze      Schwänze
7       Schwämme      Schwämme
8          Kegel         Kegel
9         Geckos        Geckon
10       Trauben       Trauben
11        Pfeile        Pfeile
12       Stollen       Stollen
13  Scheinwerfer  Scheinwerfer
14     Köchinnen     Köchinnen
15    Vorsprünge    Vorsprünge
16        Salons        Salons
17      Umstände      Umstände
18     Schlingel     Schlingel
19  Markierungen  Markierungen
1 - 243: 'Bezug N;NOM;MASC;PL' -> 'Bezüge'


Attributing with input_x_gradient...: 100%|██████████| 9/9 [00:00<00:00, 18.15it/s]


,B,e,z,Ã,¼,g,e,</s>
B,0.143,0.117,0.06,0.054,0.04,0.042,0.061,0.063
e,0.105,0.122,0.088,0.068,0.062,0.037,0.09,0.081
z,0.03,0.075,0.156,0.066,0.076,0.042,0.036,0.035
u,0.029,0.051,0.065,0.107,0.066,0.083,0.052,0.027
g,0.022,0.021,0.032,0.06,0.071,0.083,0.022,0.078
,0.058,0.034,0.027,0.061,0.027,0.028,0.026,0.058
N,0.017,0.034,0.02,0.018,0.018,0.035,0.017,0.027
;,0.04,0.039,0.03,0.044,0.035,0.043,0.048,0.049
N,0.02,0.018,0.016,0.014,0.012,0.018,0.017,0.022
O,0.047,0.03,0.033,0.024,0.029,0.016,0.022,0.009




2 - 251: 'Hinterkopf N;NOM;MASC;PL' -> 'Hinterköpfe'


Attributing with input_x_gradient...: 100%|██████████| 14/14 [00:00<00:00, 17.34it/s]


...

Attributing with input_x_gradient...: 100%|██████████| 11/11 [00:00<00:00, 17.05it/s]


,A,b,e,n,t,e,u,e,r,</s>
A,0.163,0.066,0.065,0.03,0.079,0.039,0.036,0.059,0.043,0.046
b,0.186,0.15,0.056,0.034,0.077,0.063,0.028,0.023,0.024,0.033
e,0.023,0.019,0.045,0.025,0.02,0.037,0.033,0.025,0.044,0.026
n,0.013,0.077,0.039,0.054,0.036,0.023,0.062,0.019,0.03,0.039
t,0.058,0.147,0.165,0.22,0.155,0.203,0.222,0.167,0.166,0.03
e,0.008,0.008,0.013,0.018,0.014,0.053,0.019,0.028,0.013,0.025
u,0.016,0.013,0.016,0.029,0.028,0.023,0.073,0.021,0.056,0.044
e,0.027,0.022,0.012,0.064,0.039,0.019,0.017,0.036,0.026,0.051
r,0.017,0.013,0.04,0.051,0.018,0.031,0.021,0.018,0.019,0.031
,0.028,0.035,0.012,0.013,0.018,0.038,0.032,0.014,0.024,0.03




10 - 578: 'Hauptstraße N;NOM;FEM;PL' -> 'Hauptstraßen'


Attributing with input_x_gradient...: 100%|██████████| 15/15 [00:00<00:00, 17.27it/s]


,H,a,u,p,t,s,t,r,a,Ã,,e,n,</s>
H,0.064,0.08,0.085,0.034,0.044,0.033,0.05,0.031,0.06,0.021,0.02,0.036,0.076,0.023
a,0.071,0.05,0.056,0.097,0.121,0.054,0.017,0.015,0.041,0.042,0.049,0.042,0.013,0.018
u,0.036,0.052,0.113,0.077,0.051,0.04,0.063,0.075,0.069,0.011,0.083,0.045,0.011,0.068
p,0.018,0.037,0.043,0.039,0.028,0.085,0.02,0.017,0.025,0.021,0.017,0.018,0.012,0.019
t,0.012,0.024,0.041,0.039,0.025,0.022,0.039,0.023,0.025,0.034,0.05,0.017,0.015,0.011
s,0.009,0.009,0.015,0.021,0.046,0.127,0.059,0.019,0.058,0.012,0.035,0.009,0.026,0.042
t,0.039,0.049,0.021,0.042,0.059,0.021,0.103,0.04,0.082,0.026,0.026,0.023,0.029,0.054
r,0.108,0.077,0.038,0.01,0.036,0.062,0.061,0.036,0.029,0.069,0.022,0.049,0.023,0.032
a,0.052,0.01,0.018,0.054,0.035,0.019,0.015,0.021,0.063,0.044,0.022,0.033,0.01,0.019
Ã,0.013,0.033,0.062,0.029,0.013,0.021,0.041,0.03,0.02,0.261,0.04,0.034,0.03,0.037


## Turkish

### Setup

In [8]:
model_tur_finetuned = "./models/finetuned_tur"
inseq_tur_finetuned = InseqAttributer(model_tur_finetuned, "input_x_gradient")

### Turkish Finetuned

The sample_runner function splits the given amount of samples in `two` (default is 10) so that `~50% of the samples are samples of correct predictions`, and `~50% of the samples are samples of incorrect predictions`.

First the incorrect predictions are shown, then the correct ones are shown (in the outcomes below, this translates to `5 incorrect items or 1-5, followed by 5 correct items or 6-10`).

In [11]:
sample_runner(data_sampler('./data/tur.gold', model_tur_finetuned), inseq_tur_finetuned)

                      Expected                       Predicted
0        satıyor olmalı mıydık           satıyor olmalı mıydın
1                hamızlarından                  hamızlarınızda
2                  ovmaz mıydı          ovuyor olmamalı mıydın
3     yıpranmamalı mıymışsınız         yıpranmış olacak mıydık
4                   hüzünlerde                    hüzünlerinde
5                manivelanızda              manivelalarınızdan
6            dokuyor olmalısın            dokumuş olacak mıydı
7         barışmamalılar mıydı       barışıyor olmamalı mıydık
8      sahipli yazılımlarımızı             sahipli yazılımızda
9            yanıt vermedim mi           yanıt vermemeli miydi
10       onayacak olacak mısın                 onamamış mıydık
11          tıklamış olacaktın               tıklamamış mıydın
12              prostatınızdan                prostatlarınızda
13                damlarlar mı               damlamamış mıydık
14  hareket ediyor olacaktınız         hareket etmemeli

Attributing with input_x_gradient...: 100%|██████████| 26/26 [00:01<00:00, 17.56it/s]


,a,t,Ä,±,l,m,Ä,±,Å,,,o,l,m,a,y,a,c,a,k,s,Ä,±,n,</s>
a,0.04,0.035,0.039,0.022,0.029,0.019,0.046,0.032,0.033,0.025,0.023,0.034,0.009,0.013,0.026,0.021,0.015,0.01,0.039,0.029,0.022,0.036,0.023,0.014,0.021
t,0.082,0.054,0.038,0.034,0.055,0.028,0.029,0.025,0.03,0.024,0.038,0.042,0.025,0.03,0.029,0.023,0.024,0.011,0.03,0.012,0.01,0.01,0.017,0.013,0.017
Ä,0.038,0.048,0.076,0.04,0.04,0.048,0.026,0.021,0.029,0.033,0.02,0.019,0.016,0.016,0.028,0.032,0.015,0.014,0.009,0.016,0.028,0.02,0.013,0.029,0.019
±,0.052,0.042,0.071,0.048,0.044,0.034,0.075,0.035,0.043,0.018,0.016,0.021,0.014,0.024,0.02,0.018,0.02,0.022,0.008,0.027,0.014,0.021,0.02,0.013,0.018
l,0.061,0.066,0.063,0.07,0.055,0.029,0.022,0.02,0.033,0.025,0.032,0.015,0.029,0.029,0.03,0.026,0.012,0.015,0.012,0.025,0.011,0.018,0.019,0.018,0.022
m,0.015,0.017,0.017,0.02,0.024,0.039,0.036,0.014,0.019,0.014,0.016,0.013,0.014,0.019,0.034,0.026,0.025,0.012,0.008,0.016,0.011,0.011,0.018,0.014,0.01
a,0.015,0.029,0.027,0.02,0.025,0.017,0.028,0.017,0.019,0.017,0.015,0.022,0.016,0.01,0.037,0.02,0.024,0.014,0.014,0.016,0.008,0.011,0.014,0.007,0.012
k,0.035,0.038,0.025,0.022,0.03,0.057,0.037,0.03,0.021,0.028,0.032,0.023,0.013,0.018,0.029,0.026,0.025,0.017,0.019,0.023,0.021,0.02,0.019,0.02,0.018
,0.057,0.043,0.031,0.024,0.036,0.035,0.017,0.031,0.017,0.016,0.033,0.01,0.014,0.01,0.025,0.045,0.014,0.027,0.011,0.02,0.017,0.017,0.019,0.011,0.01
V,0.013,0.012,0.013,0.009,0.015,0.012,0.01,0.008,0.008,0.007,0.011,0.007,0.007,0.009,0.012,0.011,0.006,0.006,0.008,0.012,0.009,0.008,0.01,0.007,0.01




2 - 750: 'katılmak V;OBLIG;SG;3;NEG;PRS;INTR' -> 'katılmameli mi'


Attributing with input_x_gradient...: 100%|██████████| 17/17 [00:00<00:00, 17.21it/s]


...

Attributing with input_x_gradient...: 100%|██████████| 14/14 [00:00<00:00, 17.59it/s]


,i,l,k,e,l,e,r,i,n,d,e,n,</s>
i,0.073,0.1,0.145,0.167,0.069,0.035,0.079,0.081,0.07,0.141,0.032,0.033,0.145
l,0.198,0.082,0.092,0.059,0.104,0.077,0.071,0.065,0.037,0.066,0.028,0.089,0.038
k,0.051,0.077,0.063,0.044,0.056,0.06,0.074,0.089,0.021,0.045,0.076,0.029,0.088
e,0.12,0.058,0.132,0.148,0.072,0.024,0.03,0.037,0.069,0.026,0.035,0.075,0.058
,0.045,0.043,0.018,0.029,0.143,0.062,0.04,0.032,0.031,0.018,0.022,0.019,0.019
N,0.029,0.047,0.073,0.047,0.031,0.052,0.031,0.048,0.018,0.019,0.047,0.017,0.02
;,0.046,0.077,0.029,0.034,0.044,0.027,0.034,0.024,0.046,0.027,0.025,0.02,0.028
A,0.016,0.032,0.024,0.015,0.024,0.016,0.016,0.018,0.01,0.012,0.014,0.007,0.014
B,0.032,0.072,0.057,0.059,0.043,0.03,0.014,0.013,0.03,0.017,0.01,0.015,0.006
L,0.013,0.028,0.066,0.051,0.011,0.013,0.072,0.07,0.051,0.043,0.033,0.014,0.017




10 - 1564: 'seçilmek V;OBLIG;SG;1;NEG;PST;INTR;LGSPEC03' -> 'seçilmemeli miymişim'


Attributing with input_x_gradient...: 100%|██████████| 24/24 [00:01<00:00, 16.99it/s]


,s,e,Ã,§,i,l,m,e,m,e,l,i,,m,i,y,m,i,Å,,i,m,</s>
s,0.023,0.05,0.055,0.052,0.023,0.022,0.017,0.018,0.01,0.01,0.008,0.016,0.021,0.012,0.009,0.01,0.01,0.005,0.009,0.024,0.005,0.012,0.011
e,0.026,0.073,0.039,0.031,0.025,0.03,0.044,0.014,0.015,0.01,0.007,0.012,0.02,0.016,0.005,0.015,0.014,0.006,0.017,0.01,0.007,0.01,0.008
Ã,0.019,0.033,0.145,0.036,0.037,0.048,0.018,0.021,0.015,0.014,0.02,0.031,0.031,0.019,0.01,0.012,0.034,0.017,0.014,0.018,0.015,0.022,0.031
§,0.017,0.047,0.046,0.069,0.069,0.057,0.029,0.041,0.026,0.026,0.022,0.021,0.022,0.024,0.022,0.028,0.019,0.017,0.027,0.011,0.013,0.021,0.02
i,0.021,0.027,0.033,0.05,0.053,0.028,0.031,0.023,0.02,0.019,0.029,0.008,0.008,0.02,0.01,0.023,0.009,0.006,0.009,0.006,0.014,0.025,0.009
l,0.014,0.015,0.069,0.019,0.074,0.041,0.039,0.015,0.018,0.013,0.018,0.012,0.009,0.01,0.007,0.009,0.019,0.007,0.02,0.023,0.019,0.026,0.007
m,0.007,0.013,0.021,0.022,0.025,0.024,0.063,0.038,0.036,0.031,0.019,0.013,0.014,0.014,0.018,0.013,0.009,0.012,0.013,0.008,0.014,0.017,0.016
e,0.007,0.019,0.015,0.018,0.021,0.021,0.014,0.028,0.024,0.022,0.021,0.017,0.014,0.018,0.012,0.011,0.008,0.006,0.01,0.008,0.009,0.011,0.006
k,0.019,0.033,0.05,0.058,0.052,0.036,0.027,0.059,0.028,0.037,0.026,0.02,0.022,0.026,0.032,0.03,0.033,0.018,0.022,0.012,0.016,0.025,0.024
,0.024,0.035,0.029,0.02,0.02,0.025,0.018,0.02,0.041,0.016,0.032,0.017,0.019,0.018,0.015,0.018,0.021,0.009,0.019,0.011,0.013,0.014,0.012
